In [3]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import torch
import scipy.io as sio
import os
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import torch.nn as nn

In [4]:
device = (
    "cuda"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

Using cpu device
Is CUDA supported by this system? True
CUDA version: 11.7
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce GTX 1650


In [5]:
# with open('/Anne/Facultate/licenta/ScienceDirect_files_16Dec2022_10-38-19/CSV\sig10Hz_V1_A.csv') as f:
#     train = pd.read_csv(f)
# train = train.drop('ECG', axis=1)
# train.head(5120)

# with open('/Anne/Facultate/licenta/ScienceDirect_files_16Dec2022_10-38-19/CSV\sig10Hz_V1_B.csv') as f:
#     test = pd.read_csv(f)
# # test = test.drop('ECG', axis=1)
# test.head(5120)


In [6]:
# new_train = list(train.columns)
# train.columns = new_train

# fig, axes = plt.subplots(nrows=22, ncols=1, figsize=(20, 40), sharey = True)

# for i, col in enumerate(train.columns):
#   ax = axes[i]
#   ax.plot(train[col])
#   ax.set_title(col)

# plt.tight_layout()
# # plt.show
# plt.savefig("pictures/EEG1.jpg")

In [8]:
directory = '/Anne/Facultate/licenta/Dataset/CSV'
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        with open(os.path.join(directory, filename)) as f:
            file_name = os.path.join(directory, filename)
            # print (file_name)
#5120 DATA POINTS - 20s duration of 7 photic stimulus and resting period of 10s in between stimuli freq
column_names = ["FP1", "FP2", "PG1", "PG2", "F7", "F3", "F2", "F4", "F8", "C3", "C2", "C4", "P3", "P4", "P2", "T3", "T4", "T5", "T6", "O1", "OZ", "O2", "ECG"]


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(p=0.25)
        self.fc1 = nn.Linear(4608, 128)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = nn.functional.sigmoid(x)
        return x

PermissionError: [Errno 13] Permission denied: '/'